In [ ]:
import os
from dotenv import load_dotenv

from typing import Final

import mlflow
import mlflow.entities
import mlflow.data.pandas_dataset
from mlflow.data.sources import (
    LocalArtifactDatasetSource
)

import pandas as pd
import numpy as np

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D

from sklearn.decomposition import PCA
from sklearn.preprocessing import (
    MinMaxScaler,
    StandardScaler
)

import pickle

import PIL

In [ ]:
path = "../data/raw/spectr/СКЗУ_СПЕКТР.csv"

In [ ]:
df = pd.read_csv(path, skiprows=16, sep=';')
df.drop(columns=df.columns[[0, 1]], inplace=True)

# Dataset w/o dates
column_names = pd.to_datetime(df.iloc[0, ::2].tolist(), format='%d.%m.%Y %H:%M:%S') 
df = df.iloc[:, 1::2]
df.columns = column_names
df = df.sort_index(axis=1)

df = df.tail(df.shape[0]-1)

In [ ]:
string_columns = df.select_dtypes(include='object').columns

df = df[string_columns].apply(lambda x: pd.to_numeric(x.str.replace(',', '.'), errors='coerce'))

In [ ]:
result = df.groupby(df.index // 2).sum()

In [ ]:
t = np.arange(2225)  # временная ось
f = np.arange(15097)  # частотная ось
T, F = np.meshgrid(t, f)  # создаем сетку времени и частоты
S = np.sin(2 * np.pi * F * T) 

# Построение 3D графика
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Построение поверхности
mycmap = plt.get_cmap('gist_earth')
surf = ax.plot_surface(T, F, result.values, cmap=mycmap)
ax.set_xlabel('Time')
ax.set_ylabel('Frequency')
ax.set_zlabel('Magnitude')
ax.set_title('Time vs Freq')
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
df.shape

In [ ]:
cut = df.apply(lambda x: np.mean(x.to_numpy().reshape(-1, 102), axis=1), axis=0)

In [ ]:
cut.to_csv("../data/processed/spectr/RMSA.csv", index=False)

Lowpass filtering